# Learning of Process Representations Using Recurrent Neural Networks

In [1]:
import numpy as np
import pandas as pd

from replearn.eventlog import EventLog

from replearn.embedding_predict import EmbeddingPredict
from replearn.autoencoder import AutoencoderRepresentation
from replearn.doc2vec import Doc2VecRepresentation

from replearn.clustering import Clustering

## Load event log

In [2]:
# event log configuration
event_log_path = '../logs/'
file_name = 'huge_500_10_20_5_1_1-0.0-1.json.gz'

case_attributes = None # auto-detect attributes
event_attributes = ['concept:name', 'user'] # use activity name and user
true_cluster_label = 'cluster'

# load file
event_log = EventLog(file_name, case_attributes=case_attributes, event_attributes=event_attributes, true_cluster_label=true_cluster_label)
event_log.load(event_log_path + '/' + file_name, False)
event_log.preprocess()

## Representation Learning

In [3]:
# hyperparameters
n_epochs = 10
n_batch_size = 64
n_clusters = 5

vector_size = 32

### Trace2vec

In [4]:
doc2vec = Doc2VecRepresentation(event_log)
doc2vec.build_model(append_case_attr=False, append_event_attr=False, vector_size=vector_size, concat=True, epochs=n_epochs)
doc2vec.fit()

In [5]:
# infer the vector from the model
feature_vector = doc2vec.predict(epochs=50)

In [6]:
# cluster feature vector
cluster_analysis = Clustering(event_log)
cluster_analysis.cluster(feature_vector, 'agglomerative', n_clusters, 'cosine')

cluster_result = cluster_analysis.evaluate()

In [7]:
print('Adjusted Rand Index: ' + str(cluster_result[0]))
print('Normalized Mutual Information: ' + str(cluster_result[1]))
print('F1-BCubed: ' + str(cluster_result[2]))

Adjusted Rand Index: 0.5345579223707081
Normalized Mutual Information: 0.7040911745892691
F1-BCubed: 0.7377814266285847


### Case2vec (event)

In [8]:
# train doc2vec model
doc2vec = Doc2VecRepresentation(event_log)
doc2vec.build_model(append_case_attr=False, append_event_attr=True, vector_size=vector_size, concat=True, epochs=n_epochs)
doc2vec.fit()

In [9]:
# infer the vector from the model
feature_vector = doc2vec.predict(epochs=50)

In [10]:
# cluster feature vector
cluster_analysis = Clustering(event_log)
cluster_analysis.cluster(feature_vector, 'agglomerative', n_clusters, 'cosine')

cluster_result = cluster_analysis.evaluate()

In [11]:
print('Adjusted Rand Index: ' + str(cluster_result[0]))
print('Normalized Mutual Information: ' + str(cluster_result[1]))
print('F1-BCubed: ' + str(cluster_result[2]))

Adjusted Rand Index: 0.1358345848167099
Normalized Mutual Information: 0.3169679371256953
F1-BCubed: 0.5112889799512682


### Case2vec (event+case)

In [12]:
doc2vec = Doc2VecRepresentation(event_log)
doc2vec.build_model(append_case_attr=True, append_event_attr=True, vector_size=vector_size, concat=True, epochs=n_epochs)
doc2vec.fit()

In [13]:
# infer the vector from the model
feature_vector = doc2vec.predict(epochs=50)

In [14]:
# cluster feature vector
cluster_analysis = Clustering(event_log)
cluster_analysis.cluster(feature_vector, 'agglomerative', n_clusters, 'cosine')

cluster_result = cluster_analysis.evaluate()

In [15]:
print('Adjusted Rand Index: ' + str(cluster_result[0]))
print('Normalized Mutual Information: ' + str(cluster_result[1]))
print('F1-BCubed: ' + str(cluster_result[2]))

Adjusted Rand Index: 0.027220195069314026
Normalized Mutual Information: 0.28778130557423265
F1-BCubed: 0.5255266889509068
